In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 129.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 148.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.4
    Uninstalling huggingface-hub-0.33.4:
      Successfully uninstalled huggingface-hub-0.33.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfully uninstalled transformers-4.53.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/DL_project/

/content/drive/MyDrive/DL_project


In [ ]:
import os
import pickle

import torch
from torch.utils.data import Dataset, DataLoader

from PIL import Image

from tqdm import tqdm_notebook, tqdm

from torch.optim import AdamW
from transformers import ViTImageProcessor, AutoTokenizer, VisionEncoderDecoderModel
from transformers.utils import logging

In [ ]:
# --- Configuration ---
IMG_DIR = r'./Flicker8k_images'
CAP_DIR = r'./Flicker8k_captions'
ENCODER_ID = "google/vit-base-patch16-224-in21k"
DECODER_ID = "gpt2"
TOKENIZER_NAME = "gpt2"
SAVED_MODEL_PATH = r'./saved-model' # Path to load initial model
CHECKPOINT_DIR = r'./image-captioning-model' # Directory to save new checkpoints

MAX_LEN = 48
BATCH_SIZE = 32 # Increased to match your original DataLoader
EPOCHS = 5
LR = 1e-5
gradient_acc_steps = 8
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
with open(os.path.join(CAP_DIR, 'train_data.pickle'), 'rb') as f:
    train_data = pickle.load(f)

# Load dev_data
with open(os.path.join(CAP_DIR, 'dev_data.pickle'), 'rb') as f:
    dev_data = pickle.load(f)

# Load test_data
with open(os.path.join(CAP_DIR, 'test_data.pickle'), 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
class Flickr8kDataset(Dataset):
    def __init__(self, data, tokenizer, img_processor, img_dir, max_len):
        self.tokenizer = tokenizer
        self.processor = img_processor
        self.img_dir = img_dir
        self.max_len = max_len
        self.data = []

        # Use all captions for better training
        for filename, captions in data.items():
            for cap in captions:
                self.data.append((filename, cap))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filename, caption = self.data[idx]

        # --- Image Processing ---
        img_path = os.path.join(self.img_dir, filename)
        img = Image.open(img_path).convert("RGB")
        pixel_values = self.processor(img, return_tensors='pt').pixel_values.squeeze(0)

        # --- Caption Processing ---
        tokenized_output = self.tokenizer(
            caption,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        tokens = tokenized_output.input_ids.squeeze(0)
        attention_mask = tokenized_output.attention_mask.squeeze(0)

        labels = tokens.clone()
        labels[labels == cap_tokenizer.pad_token_id] = -100

        return {'pixel_values': pixel_values, 'labels': labels, 'attention_mask': attention_mask, 'filename': filename}

In [ ]:
img_processor = ViTImageProcessor.from_pretrained(ENCODER_ID)
cap_tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
cap_tokenizer.add_special_tokens({'pad_token': '<PAD>', 'bos_token': '<BOS>'})

train_dataset = Flickr8kDataset(train_data, cap_tokenizer, img_processor, IMG_DIR, MAX_LEN)
dev_dataset = Flickr8kDataset(dev_data, cap_tokenizer, img_processor, IMG_DIR, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
'''
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    ENCODER_ID, DECODER_ID
)

# Set the dropout for the encoder
model.encoder.config.attention_probs_dropout_prob = 0.2
model.encoder.config.hidden_dropout_prob = 0.2

# Set the dropout for the decoder
model.decoder.config.attention_probs_dropout_prob = 0.2
model.decoder.config.hidden_dropout_prob = 0.2

# Set the generation params on the main config (as you've done)
model.config.pad_token_id = cap_tokenizer.pad_token_id
model.config.decoder_start_token_id = cap_tokenizer.bos_token_id
model.config.eos_token_id = cap_tokenizer.eos_token_id
model.decoder.resize_token_embeddings(len(cap_tokenizer))

model.to(DEVICE)
#model.save_pretrained(SAVED_MODEL_PATH)
'''




"\nmodel = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(\n    ENCODER_ID, DECODER_ID\n)\n\n# Set the dropout for the encoder\nmodel.encoder.config.attention_probs_dropout_prob = 0.2\nmodel.encoder.config.hidden_dropout_prob = 0.2\n\n# Set the dropout for the decoder\nmodel.decoder.config.attention_probs_dropout_prob = 0.2\nmodel.decoder.config.hidden_dropout_prob = 0.2\n\n# Set the generation params on the main config (as you've done)\nmodel.config.pad_token_id = cap_tokenizer.pad_token_id\nmodel.config.decoder_start_token_id = cap_tokenizer.bos_token_id\nmodel.config.eos_token_id = cap_tokenizer.eos_token_id\nmodel.decoder.resize_token_embeddings(len(cap_tokenizer))\n\nmodel.to(DEVICE)\n#model.save_pretrained(SAVED_MODEL_PATH)\n"

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained(r'./image-captioning-model/epoch_8')
model.to(DEVICE)


VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (inte

In [ ]:
def evaluate(model, loader, tokenizer, device, max_len):
    """
    Evaluates the model on a given dataset.
    Calculates loss and shows some generated captions, omitting the BLEU calculation.
    """
    model.eval()
    total_loss = 0
    generated_captions = {}

    with torch.no_grad():
        for i, batch in enumerate(tqdm(loader, desc="Evaluating")):
            # if i==1: break
            pixel_values = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)
            decoder_attention_mask = batch['attention_mask'].to(device)
            filenames = batch.get('filename')

            outputs = model(
                pixel_values=pixel_values,
                labels=labels,
                decoder_attention_mask=decoder_attention_mask
            )

            total_loss += outputs.loss.item()
            logging.disable_progress_bar()

            logging.enable_progress_bar()

            if i == 0:
              generated_ids = model.generate(
                  pixel_values=pixel_values,
                  max_new_tokens=20,
                  decoder_start_token_id=cap_tokenizer.bos_token_id,
                  pad_token_id=cap_tokenizer.pad_token_id,
                  eos_token_id=cap_tokenizer.eos_token_id,
                  do_sample=True,
                  top_p=0.9,
                  temperature=0.7,
                  #num_beams=5,
                  #length_penalty=3.0,
                  repetition_penalty=3.0,
                  min_length=5,
                  early_stopping=True
              )

              preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
              for j, filename in enumerate(filenames):
                  if filename not in generated_captions:
                      generated_captions[filename] = preds[j]
            # ---------------------------------

    avg_loss = total_loss / len(loader)

    # Print some example captions
    print("\n--- Example Generated Captions ---")
    for filename, caption in list(generated_captions.items())[:10]: # first 3 examples
        print(f"Image: {filename}\nCaption: {caption}\n")
    print("--------------------------------\n")

    return avg_loss


In [ ]:
'''
# STEP 1: train cross attention layer only (5 epochs)
params_to_train = []

for name, param in model.named_parameters():
    # Check if the parameter is part of the cross-attention or its LayerNorm
    if "crossattention" in name or "ln_cross_attn" in name:
        param.requires_grad = True
        params_to_train.append(param)
    else:
        # Freeze all other parameters
        param.requires_grad = False
optimizer = AdamW(params_to_train, lr=LR)
'''

# STEP 2: train decoder only - exclude cross attention (3 epochs)
for param in model.parameters():
    param.requires_grad = False

# Unfreeze decoder (excluding cross-attention)
for name, param in model.decoder.named_parameters():
    # Only unfreeze if it's NOT part of cross-attention or its LayerNorm
    if "crossattention" not in name and "ln_cross_attn" not in name:
        param.requires_grad = True

# Create a list of parameters that actually require gradients FROM THE DECODER ONLY
params_to_train = [p for p in model.decoder.parameters() if p.requires_grad]

# 4. Initialize the optimizer with ONLY the trainable decoder parameters
optimizer = AdamW(params_to_train, lr=1e-4) # Or your desired LR for stage two


#optimizer = AdamW([
#    {'params': model.encoder.parameters(), 'lr': 1e-6},   # ViT
#    {'params': model.decoder.parameters(), 'lr': 5e-5},   # GPT-2
#])

# --- Training & Evaluation Loop ---
print(f"Starting training on {DEVICE}...")
for epoch in range(0,10):
    model.train()
    total_loss = 0

    print(f"Epoch {epoch+1}/{EPOCHS}")
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(tqdm(train_loader, desc="Training Batches")):
        pixel_values = batch['pixel_values'].to(DEVICE)
        attn_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        outputs = model(pixel_values=pixel_values, labels=labels, decoder_attention_mask=attn_mask)
        loss = outputs.loss
        total_loss += loss.item()

        loss = loss / gradient_acc_steps
        loss.backward()

        if (batch_idx + 1) % gradient_acc_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    # Optional: Handle the remainder (if len(train_loader) is not a multiple of gradient_acc_steps)
    if (batch_idx + 1) % gradient_acc_steps != 0:
        optimizer.step()
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Train Loss: {avg_train_loss:.4f}")

    # --- Save Model ---
    epoch_output_dir = os.path.join(CHECKPOINT_DIR, f"epoch_{epoch+1}")
    os.makedirs(epoch_output_dir, exist_ok=True)
    model.save_pretrained(epoch_output_dir)
    cap_tokenizer.save_pretrained(epoch_output_dir)
    print(f"Model saved to {epoch_output_dir}")

    # --- Evaluation ---
    avg_dev_loss = evaluate(
        model=model,
        loader=dev_loader,
        tokenizer=cap_tokenizer,
        device=DEVICE,
        max_len=MAX_LEN
    )
    print(f"Dev Loss: {avg_dev_loss:.4f}")

print("Training finished!")

Starting training on cuda...
Epoch 9/5


Training Batches: 100%|██████████| 938/938 [1:19:17<00:00,  5.07s/it]


Train Loss: 2.9322
Model saved to ./image-captioning-model/epoch_9


Evaluating: 100%|██████████| 157/157 [01:43<00:00,  1.52it/s]



--- Example Generated Captions ---
Image: 2090545563_a4e66ec76b.jpg
Caption:  little wearing hat on bench a boy girl blue black white and striped . in park , bags her .

Image: 3393035454_2d2370ffd4.jpg
Caption:  man a wearing jacket blue on leash from tree high grass forest area night sky background light background foliage trees

Image: 3695064885_a6922f06b2.jpg
Caption:  black white brown jumping a and pole woman purple . dog ananis on course spectators are background with

Image: 1679557684_50a206e4a9.jpg
Caption:  small dog to a with in mouth large and fence wood area two dogs outdoors camera out frame big .

Image: 3582685410_05315a15b8.jpg
Caption:  girls on beach holding while . person red off ground to in of . , and man sunglasses theng

Image: 1579798212_d30844b4c5.jpg
Caption:  boy out the . at waters with toy mouth and in ofaker bubbles su coming from bottom of

Image: 269650644_059a84ece5.jpg
Caption:  people ice on sk . in rink adults children skating the . ball the ofsk

Training Batches: 100%|██████████| 938/938 [13:04<00:00,  1.19it/s]


Train Loss: 2.8525
Model saved to ./image-captioning-model/epoch_10


Evaluating: 100%|██████████| 157/157 [01:43<00:00,  1.52it/s]


--- Example Generated Captions ---
Image: 2090545563_a4e66ec76b.jpg
Caption:  girl on knees her in of and with armsstretched she to chest sitting the .ed . child a

Image: 3393035454_2d2370ffd4.jpg
Caption:  boy in jacket red jumping tree . on , man blue white through . and sky background foliage trees the

Image: 3695064885_a6922f06b2.jpg
Caption:  dog over hurdle fence onlook . on , and in of ' outdoor area dogs at . line course with

Image: 1679557684_50a206e4a9.jpg
Caption:  white brown jumping a ,- dog . the one holding ball standing its straight is on grass front trees

Image: 3582685410_05315a15b8.jpg
Caption:  man woman on beach in bikini , to and at . from of she sit a with boy the is

Image: 1579798212_d30844b4c5.jpg
Caption:  baby a water . in , boys gets face with of . sponge mud spl from . held the

Image: 269650644_059a84ece5.jpg
Caption:  group sk traveling white through .-ed skating . area nighttime people iceates the in and out

--------------------------------

Dev L